# Manipulation of quantum states in QuAIRKit

A pure quantum state is a unit vector in a Hilbert space $\mathcal{H}$. A mixed quantum state could be described by a density matrix $\rho$, which is a Hermitian, positive semi-definite matrix with unit trace. Concretely, pure states are of the form $|\psi \rangle \langle \psi |$ where $|\psi \rangle \in \mathcal{H}$ is a normalized vector. Mixed state is the convex combination of the pure states $\sum_{x \in \mathcal{X}}p(x)|\psi_x \rangle \langle \psi_x |$ for some set $\{|\psi_x \rangle\}_{x \in \mathcal{X}}$ of state vectors defined with respect to a finite alphabet $\mathcal{X}$, where $p: \mathcal{X} \rightarrow [0, 1]$ is a probability distribution. The `State` class in QuairKit supports operations related to quantum states, mainly including the preparation of quantum states and operations on them.

**Table of Contents**

- [Preparation of states](#Preparation-of-states)
- [Information of a State instance](#Information-of-a-State-instance)
- [Manipulation of states](#Manipulation-of-states)
- [Interaction with environments](#Interaction-with-environments)

In [1]:
import torch

import quairkit as qkit
from quairkit import Hamiltonian, to_state
from quairkit.qinfo import *
from quairkit.database import *

## Preparation of states

In QuAIRKit, states are prepared in several ways. For some commonly-used states, they are available in QuAIRKit database:


In [2]:
num_qubits = 2  # set the number of qubits

state = zero_state(num_qubits)  # |00>
print(f"zero states with 2 qubits: {state}")

state = bell_state(num_qubits)  # (|00> + |11>) / sqrt(2)
print(f"Bell states: {state}")

state = isotropic_state(num_qubits, prob=0.1)  # isotropic state
print(f"isotropic state: {state}")

zero states with 2 qubits: 
-----------------------------------------------------
 Backend: state_vector
 System dimension: [2, 2]
 System sequence: [0, 1]
[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
-----------------------------------------------------

Bell states: 
-----------------------------------------------------
 Backend: state_vector
 System dimension: [2, 2]
 System sequence: [0, 1]
[0.71+0.j 0.  +0.j 0.  +0.j 0.71+0.j]
-----------------------------------------------------

isotropic state: 
-----------------------------------------------------
 Backend: density_matrix
 System dimension: [2, 2]
 System sequence: [0, 1]
[[0.27+0.j 0.  +0.j 0.  +0.j 0.05+0.j]
 [0.  +0.j 0.22+0.j 0.  +0.j 0.  +0.j]
 [0.  +0.j 0.  +0.j 0.22+0.j 0.  +0.j]
 [0.05+0.j 0.  +0.j 0.  +0.j 0.27+0.j]]
-----------------------------------------------------



Users can also prepare a random state, with desirable size and rank. Additionally, by setting `is_real=True` one can restrict the random state within the real space.

In [3]:
state = random_state(num_qubits, rank=1)  # random 2-qubit pure state
state = random_state(num_qubits, is_real=True)  # random 2-qubit real state
state = random_state(num_qubits, size=1000)  # 1000 random 2-qubit states

Another way of preparing a quantum state uses the function `to_state`. `to_state` converts a *torch.Tensor* or a *numpy.ndarray* data type to a `State` instance. Allowed shapes of the input tensor are listed in the following table. Based on the input data, the state will be represented differently. If the input is a vector, the state will be represented by a state vector, whereas if the input is a density matrix, the state will be adapted to a density matrix. The type can be checked by the `state.backend`.

|                | single  state       | batch  states       |
|----------------|:---------------------:|:---------------------:|
| state vector   | [d], [1, d], [d, 1] | [d1, ..., dn, d, 1] |
| density matrix | [d, d]              | [d1, ..., dn, d, d] |

In [4]:
data = haar_state_vector(
    num_qubits
)  # randomly generate a state vector with 2 qubits following Haar random
state = to_state(data)
print(f"A state vector with 2 qubits following Haar random{state}")

data = random_density_matrix(num_qubits)  # random 2-qubit density matrix
state = to_state(data)
print(f"type of the state: {state.backend}")

A state vector with 2 qubits following Haar random
-----------------------------------------------------
 Backend: state_vector
 System dimension: [2]
 System sequence: [0]
[-0.26-0.11j  0.74+0.61j]
-----------------------------------------------------

type of the state: density_matrix


## Information of a State instance
The information of a `State` instance is provided as follows. Three random generated single-qubit states $\{ |\psi_j\rangle \}_{j=1}^3$ are taken as examples.

In [5]:
state = random_state(num_qubits=1, rank=1, size=3)  # 3 random single-qubit pure states
print(f"3 random single-qubit pure states: {state}")

3 random single-qubit pure states: 
-----------------------------------------------------
 Backend: state_vector
 System dimension: [2]
 System sequence: [0]
 Batch size: [3]

 # 0:
[0.94+0.11j 0.15+0.27j]
 # 1:
[-0.59-0.44j -0.57-0.37j]
 # 2:
[0.42-0.16j 0.88-0.16j]
-----------------------------------------------------



One can obtain the ket $\{ |\psi_j\rangle \}_{j=1}^3$, bra $\{ \langle\psi_j| \}_{j=1}^3$ and the density matrix $\{ |\psi_j\rangle\langle\psi_j| \}_{j=1}^3$ of these states. (Note that ket and bra forms are only available for pure states.)

In [6]:
print("Its density matrix is :\n", state.density_matrix)
print("\nIts ket is :\n", state.ket)
print("\nIts bra is :\n", state.bra)

Its density matrix is :
 tensor([[[0.9029+0.0000j, 0.1728-0.2404j],
         [0.1728+0.2404j, 0.0971+0.0000j]],

        [[0.5393+0.0000j, 0.4976+0.0298j],
         [0.4976-0.0298j, 0.4607+0.0000j]],

        [[0.1976+0.0000j, 0.3922-0.0685j],
         [0.3922+0.0685j, 0.8024+0.0000j]]])

Its ket is :
 tensor([[[ 0.9436+0.1123j],
         [ 0.1506+0.2727j]],

        [[-0.5900-0.4373j],
         [-0.5685-0.3708j]],

        [[ 0.4164-0.1556j],
         [ 0.8806-0.1645j]]])

Its bra is :
 tensor([[[ 0.9436-0.1123j,  0.1506-0.2727j]],

        [[-0.5900+0.4373j, -0.5685+0.3708j]],

        [[ 0.4164+0.1556j,  0.8806+0.1645j]]])


We can also use the method `numpy()` to output *numpy.ndarray* type data for `State`.

In [7]:
print("\nThe state is :\n", state.numpy())


The state is :
 [[ 0.9435761 +0.11229491j  0.15063664+0.27269506j]
 [-0.5899563 -0.4373287j  -0.5684789 -0.3708463j ]
 [ 0.41636243-0.1555816j   0.8805542 -0.16450219j]]


Other useful information can be read via `State` object.

In [8]:
print("The trace of these states are", state.trace())
print("The rank of these states are", state.rank)
print("The size of these states are", state.dim)
print("The shape of vectorization of these states are", state.vec.shape)

The trace of these states are tensor([1.+0.j, 1.+0.j, 1.+0.j])
The rank of these states are 1
The size of these states are 2
The shape of vectorization of these states are torch.Size([3, 4, 1])


In [9]:
print("The number of systems in these states are", state.num_systems)
print("Are these states qubits?", state.are_qubits())
print("Are these states qutrits?", state.are_qutrits())

The number of systems in these states are 1
Are these states qubits? True
Are these states qutrits? False


`State` instance supports direct indexing, returning the $i$-th state in the batch.

In [10]:
print(f"the second and third state in the batch: {state[1:]}")

the second and third state in the batch: 
-----------------------------------------------------
 Backend: state_vector
 System dimension: [2]
 System sequence: [0]
 Batch size: [2]

 # 0:
[-0.59-0.44j -0.57-0.37j]
 # 1:
[0.42-0.16j 0.88-0.16j]
-----------------------------------------------------



## Manipulation of states

Matrix multiplication is implemented via `@`, which makes calculating the inner product of two pure states $\langle \psi | \phi \rangle$ easily.

In [11]:
state_1 = zero_state(num_qubits=1).density_matrix
data = random_density_matrix(num_qubits=1)
state_2 = to_state(data).density_matrix
print(f"matrix multiplication:\n{state_1 @ state_2}")
print("The overlap of state_1 and state_2 is :", trace(state_1 @ state_2))

matrix multiplication:
tensor([[0.1567+0.0000j, 0.2268-0.2841j],
        [0.0000+0.0000j, 0.0000+0.0000j]])
The overlap of state_1 and state_2 is : tensor(0.1567+0.j)


User can also use `NKron` to obtain the tensor product $\rho \otimes \sigma$ of the two quantum states $\rho$ and $\sigma$.


In [12]:
product_state = NKron(state_1, state_2)
print(f"tensor product:\n{product_state}")

tensor product:
tensor([[0.1567+0.0000j, 0.2268-0.2841j, 0.0000+0.0000j, 0.0000+0.0000j],
        [0.2268+0.2841j, 0.8433+0.0000j, 0.0000+0.0000j, 0.0000+0.0000j],
        [0.0000+0.0000j, 0.0000+0.0000j, 0.0000+0.0000j, 0.0000+0.0000j],
        [0.0000+0.0000j, 0.0000+0.0000j, 0.0000+0.0000j, 0.0000+0.0000j]])


Permuting subsystems can be implemented by adapting the value of `system_seq`. However, such a change does not affect its output on ket, bra, and density matrix, in which case `system_seq` would be reset to the default sequence. Here, three randomly prepared 2-qubit states $\{ |\phi_j\rangle \}_{j=1}^3$ are permuted as examples.

In [13]:
state = random_state(num_qubits=2, rank=1, size=1)  # random 2-qubit pure states
print(f"1 random 2-qubit pure states: {state}")

state.system_seq = [1, 0]  # permutation
print(f"state after permutation: {state}")

1 random 2-qubit pure states: 
-----------------------------------------------------
 Backend: state_vector
 System dimension: [2, 2]
 System sequence: [0, 1]
[ 0.44+0.19j -0.33+0.29j  0.09+0.18j  0.72+0.16j]
-----------------------------------------------------

state after permutation: 
-----------------------------------------------------
 Backend: state_vector
 System dimension: [2, 2]
 System sequence: [1, 0]
[ 0.44+0.19j  0.09+0.18j -0.33+0.29j  0.72+0.16j]
-----------------------------------------------------



User can also clone a state, or change its data type and device.

In [14]:
print("The dtype of these states are", state.dtype)
print("The device of these states are", state.device)

new_state = state.clone().to(
    dtype=torch.complex128, device="cpu"
)  # change to "cuda" if gpu is available
print("The dtype of new states are", new_state.dtype)
print("The device of new states are", new_state.device)

The dtype of these states are torch.complex64
The device of these states are cpu
The dtype of new states are torch.complex128
The device of new states are cpu


## Interaction with environments

`State` instances can be sent to a quantum environment for further processing. For example, let the state evolve under a unitary operator.

In [15]:
unitary = random_unitary(num_qubits=1)
state_evo = state.evolve(unitary, sys_idx=[1])
print(f"state after evolving with unitary: {state_evo}")

state after evolving with unitary: 
-----------------------------------------------------
 Backend: state_vector
 System dimension: [2, 2]
 System sequence: [1, 0]
[ 0.1 -0.47j -0.69-0.02j -0.08+0.43j -0.26-0.17j]
-----------------------------------------------------



Then, apply a random quantum channel to the state. Note that when pure states are sent to noisy environments, they will be automatically converted to mixed states, in which case their ket and bra properties will be lost.

In [16]:
kraus = random_channel(num_qubits=1)
state_kra = state.transform(kraus, sys_idx=[0], repr_type="kraus")
print(f"state after transformation: {state}")

state after transformation: 
-----------------------------------------------------
 Backend: state_vector
 System dimension: [2, 2]
 System sequence: [0, 1]
[ 0.44+0.19j -0.33+0.29j  0.09+0.18j  0.72+0.16j]
-----------------------------------------------------



In [17]:
choi = random_channel(num_qubits=1, target="choi")
state_cho = state.transform(choi, sys_idx=[1], repr_type="choi")
print(f"state after transformation: {state}")

state after transformation: 
-----------------------------------------------------
 Backend: state_vector
 System dimension: [2, 2]
 System sequence: [0, 1]
[ 0.44+0.19j -0.33+0.29j  0.09+0.18j  0.72+0.16j]
-----------------------------------------------------



`State.expec_val()` calculates the expectation value of a state under a given observable. Here we set the observable to Pauli $Z$ operator on the first qubit. 

In [18]:
observable = Hamiltonian([(2, "Z0")])
print(
    "the expectation value under the given observable: ",
    state.expec_val(hamiltonian=observable),
)

the expectation value under the given observable:  tensor(-0.3195)


`State.measure()` returns the measurement result in the computational basis.

In [19]:
print("Theoretical value is :", state.measure())  # theoretical value

Theoretical value is : tensor([0.2273, 0.1929, 0.0403, 0.5396])


---

*Table: A reference of notation conventions in this tutorial.*

| Symbol        | Variant          | Description                               |
|:---------------:|:------------------:|-------------------------------------------|
| $\mathcal{H}$     | $\mathcal{H}_A$      | a Hilbert space (of quantum system $A$)   |
|$X$ | $\sigma_x$| Pauli X|
| $\vert \psi \rangle$ | $\vert \psi_j \rangle$ | the $j$-th pure state|
| $\langle \psi \vert$ |  | conjugate transpose of $\vert \psi \rangle$|
| $\rho$ | | general quantum state|
|$\langle \psi \vert \phi \rangle$ |  | inner product of two pure states|
| $\rho \otimes \sigma$ | | tensor product of two states|

In [20]:
qkit.print_info()


---------VERSION---------
quairkit: 0.4.0
torch: 2.7.0+cpu
numpy: 1.26.0
scipy: 1.14.1
matplotlib: 3.10.0
---------SYSTEM---------
Python version: 3.10.16
OS: Windows
OS version: 10.0.26100
---------DEVICE---------
CPU: ARMv8 (64-bit) Family 8 Model 1 Revision 201, Qualcomm Technologies Inc
